In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split        
from torchvision import transforms
import os
import cv2
import torch.nn.functional as F
from torchvision import models
import torch_geometric.nn as geom_nn


In [2]:
import torch
import Data_Loader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print (device)
# FER Dataset
#data_folder_path = 'FER_2013/train'
#label_mapping = {'fear': 1, 'angry': 2, 'happy': 3, 'sad': 4, 'surprise': 5, 'neutral': 6,'disgust': 7}

# CK+ Dataset
#data_folder_path = 'CK+/CK+'
#label_mapping = {'fear': 1, 'anger': 2, 'happy': 3, 'sadness': 4, 'surprise': 5, 'contempt': 6, 'disgust': 7}
#data_folder_path = '/content/drive/MyDrive/CK/FER/train'

#FERG
data_folder_path = 'FERG/FERG'
label_mapping = {'fear': 1, 'anger': 2, 'joy': 3, 'sadness': 4, 'surprise': 5, 'neutral': 6,'disgust': 7}
#FERG
#data_folder_path = 'JAFFE/jaffedbase'
#label_mapping = {'fear': 1, 'angry': 2, 'happy': 3, 'sad': 4, 'surprise': 5, 'neutral': 6,'disgust': 7}

test_size  = 0.2
val_size   = 0.1
batch_size =  18
images_per_class= -1

train_loader,test_loader,val_loader = Data_Loader.Read_Data(data_folder_path,label_mapping,device, images_per_class, test_size, val_size, batch_size)

cuda
2 -1 9169
7 -1 8571
1 -1 7419
3 -1 7330
6 -1 6939
4 -1 8827
5 -1 7510
torch.Size([55765, 3, 360, 360])


Modified model ---

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import torch_geometric.nn as geom_nn

class GCF(nn.Module):
    def __init__(self, num_classes, device, base_model='vgg16', mode='cnn+gcn'):
        super(GCF, self).__init__()
        self.mode = mode
        self.device = device
        self.base_model = base_model.lower()

        # Load the selected pretrained model and define feature size
        cnn_output_size = 0

        if self.base_model == 'vgg16':
            self.model = models.vgg16(pretrained=True)
            num_ftrs = self.model.classifier[-1].in_features
            self.model.classifier = nn.Sequential(*list(self.model.classifier.children())[:-1], nn.Flatten())
            cnn_output_size = 4096

        elif self.base_model == 'vgg19':
            self.model = models.vgg19(pretrained=True)
            num_ftrs = self.model.classifier[-1].in_features
            self.model.classifier = nn.Sequential(*list(self.model.classifier.children())[:-1], nn.Flatten())
            cnn_output_size = 4096

        elif self.base_model == 'resnet18':
            self.model = models.resnet18(pretrained=True)
            num_ftrs = self.model.fc.in_features
            self.model.fc = nn.Flatten()
            cnn_output_size = 512

        elif self.base_model == 'resnet34':
            self.model = models.resnet34(pretrained=True)
            num_ftrs = self.model.fc.in_features
            self.model.fc = nn.Flatten()
            cnn_output_size = 512

        elif self.base_model == 'resnet50':
            self.model = models.resnet50(pretrained=True)
            num_ftrs = self.model.fc.in_features
            self.model.fc = nn.Flatten()
            cnn_output_size = 2048

        elif self.base_model == 'densenet121':
            self.model = models.densenet121(pretrained=True)
            num_ftrs = self.model.classifier.in_features
            self.model.classifier = nn.Sequential(nn.Flatten())
            cnn_output_size = 1024

        elif self.base_model == 'densenet169':
            self.model = models.densenet169(pretrained=True)
            num_ftrs = self.model.classifier.in_features
            self.model.classifier = nn.Sequential(nn.Flatten())
            cnn_output_size = 1664

        elif self.base_model == 'densenet201':
            self.model = models.densenet201(pretrained=True)
            num_ftrs = self.model.classifier.in_features
            self.model.classifier = nn.Sequential(nn.Flatten())
            cnn_output_size = 1920

        elif self.base_model == 'efficientnet_b0':
            self.model = models.efficientnet_b0(pretrained=True)
            num_ftrs = self.model.classifier[-1].in_features
            self.model.classifier = nn.Flatten()
            cnn_output_size = 1280

        elif self.base_model == 'inception_v3':
            self.model = models.inception_v3(pretrained=True, aux_logits=True)
            num_ftrs = self.model.fc.in_features
            self.model.fc = nn.Flatten()
            cnn_output_size = 2048

        else:
            raise ValueError(f"Unsupported base model: {self.base_model}")

        # Define output sizes and fully connected layers
        gcn_output_size = 128
        if self.mode == "cnn" or self.mode == "cnn+gcn":
            self.fc_cnn = nn.Linear(num_ftrs, cnn_output_size)
        if self.mode == "gcn" or self.mode == "cnn+gcn":
            self.fc_gcn = nn.Linear(256, gcn_output_size)

        combined_output_size = cnn_output_size + gcn_output_size if mode == "cnn+gcn" else max(cnn_output_size, gcn_output_size)
        self.fc = nn.Linear(combined_output_size, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)

        self.dropout = nn.Dropout(0.5)
        self.gnn = self._build_gnn_layer(cnn_output_size)

    def _build_gnn_layer(self, cnn_output_size):
        return GNNLayer(in_channels=cnn_output_size, hidden_channels=256, out_channels=256, device=self.device)

    def forward(self, images):
        images = images.to(self.device)
        cnn_features = self.model(images)

        if self.mode == "cnn" or self.mode == "cnn+gcn":
            cnn_features = cnn_features.view(cnn_features.size(0), -1)
            cnn_features = self.fc_cnn(cnn_features)

        if self.mode == "gcn" or self.mode == "cnn+gcn":
            gcn_features = self.gnn(cnn_features)
            gcn_features = gcn_features.view(gcn_features.size(0), -1)
            gcn_features = self.fc_gcn(gcn_features)

        if self.mode == "cnn+gcn":
            features = torch.cat((cnn_features, gcn_features), dim=1)
        elif self.mode == "cnn":
            features = cnn_features
        else:
            features = gcn_features

        out = self.fc(features)
        out = F.relu(self.fc2(out))
        out = self.dropout(out)
        out = self.fc3(out)

        return out

class GNNLayer(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, device):
        super(GNNLayer, self).__init__()
        self.conv1 = geom_nn.GraphConv(in_channels, hidden_channels)
        self.device = device
        
        
        #VERSION 2
        #self.edge_index = torch.tensor(
        #    [[0, 1, 0, 3, 0, 4, 1, 2, 1, 4, 3, 4, 3, 6, 4, 5, 4, 7, 6, 7, 7, 8, 2, 4, 4, 6, 5, 4, 4, 8],
        #     [0, 1, 0, 3, 0, 4, 1, 2, 1, 4, 3, 4, 3, 6, 4, 5, 4, 7, 6, 7, 7, 8, 2, 4, 4, 6, 5, 4, 4, 8]],
        #    dtype=torch.long).to(self.device)
        

        # VERSION 3
        self.edge_index = torch.tensor(
            [[1, 0, 3, 0, 4, 0, 2, 1, 4, 1, 4, 3, 6, 3, 5, 4, 7, 4, 7, 6, 8, 7, 4, 2, 6, 4, 4, 5, 8, 4],
             [1, 0, 3, 0, 4, 0, 2, 1, 4, 1, 4, 3, 6, 3, 5, 4, 7, 4, 7, 6, 8, 7, 4, 2, 6, 4, 4, 5, 8, 4]],
            dtype=torch.long).to(self.device)

    def forward(self, x):
        return F.relu(self.conv1(x, self.edge_index))


In [4]:
from Training import train
import torch
import torch.optim as optim
import torch.nn as nn


def get_available_device():
    if torch.cuda.is_available():
        num_gpus = torch.cuda.device_count()
        print(f"Found {num_gpus} GPU(s) available.")
        for i in range(num_gpus):
            print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        gpu_choice = 1
        device = torch.device(f"cuda:{gpu_choice}")
        print(f"Using GPU {gpu_choice}: {torch.cuda.get_device_name(gpu_choice)}")
    else:
        device = torch.device("cpu")
        print("CUDA is not available. Using CPU.")
    return device

device = get_available_device()
#device = torch.device("cpu")
def gcf_train(mode,base):
  num_classes = 8
  GCF_model = GCF(num_classes,device,base_model=base, mode=mode).to(device)

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(GCF_model.parameters(), lr=0.00005,weight_decay=1e-4)
  num_epochs = 20
  train(GCF_model,device,train_loader,val_loader,num_epochs,criterion,optimizer)
  output_path = "D:/GCF"
  model_filename = f"FER+{mode}"
  full_output_path = os.path.join(output_path, model_filename)

  torch.save(GCF_model.state_dict(), f"Vgg16-Raf-{model_filename}.h5")
  return GCF_model


def gcf_test(GCF_model):
    GCF_model.eval()
    criterion=nn.CrossEntropyLoss()
    with torch.no_grad():
        test_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = GCF_model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()


            _, predicted = torch.max(outputs, 1)


            correct_predictions += (predicted == targets).sum().item()


            total_predictions += targets.size(0)


        avg_test_loss = test_loss / len(test_loader)


    accuracy = correct_predictions / total_predictions


    print("Test Accuracy:", accuracy)

Found 2 GPU(s) available.
GPU 0: NVIDIA GeForce RTX 4060 Ti
GPU 1: NVIDIA GeForce RTX 4060 Ti
Using GPU 1: NVIDIA GeForce RTX 4060 Ti


In [5]:
import pandas as pd

models_list = [
    'vgg16', 'vgg19', 'resnet18', 'resnet34', 'resnet50',
    'densenet121', 'densenet169', 'densenet201',
    'efficientnet_b0'
]

# To store results
results = []

# Loop through each model type, train and test
for base_model in models_list:
    print("\n" + "-"*40)
    print(f"Processing Model: {base_model.upper()}")
    print("-"*40)
    
    # Initialize, train and test the model
    GCF_model = gcf_train("cnn+gcn", base_model)
    accuracy = gcf_test(GCF_model)
    
    # Add results to the list
    results.append([base_model, "cnn+gcn", accuracy])

# Create a DataFrame from the results and display as a table
results_df = pd.DataFrame(results, columns=["Base Model", "Mode", "Accuracy (%)"])

# Save results to Excel file
results_df.to_excel("model_results.xlsx", index=False)

print("\nFinal Results:")
print(results_df.to_string(index=False))
print("\nResults have been saved to 'model_results.xlsx'")



----------------------------------------
Processing Model: VGG16
----------------------------------------


c:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\ProgramData\anaconda3\Lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, 

Epoch 1/20, Train Loss: 0.04, Train Acc: 98.65%, Val Loss: 0.00, Val Acc: 99.98%
Epoch 2/20, Train Loss: 0.01, Train Acc: 99.81%, Val Loss: 0.00, Val Acc: 100.00%
Epoch 3/20, Train Loss: 0.01, Train Acc: 99.80%, Val Loss: 0.01, Val Acc: 99.75%
Epoch 4/20, Train Loss: 0.00, Train Acc: 99.93%, Val Loss: 0.02, Val Acc: 99.69%
Epoch 5/20, Train Loss: 0.01, Train Acc: 99.85%, Val Loss: 0.00, Val Acc: 100.00%


KeyboardInterrupt: 